# Mass&Calsification detction & converting images from dicom 2 png

In [ ]:
import os
import numpy as np
import pydicom
from PIL import Image, ImageOps
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

# Apply windowing to enhance image contrast based on Window Center (WC) and Window Width (WW)
def apply_windowing(img_data, wc, ww):
    try:
        # Calculate minimum and maximum intensity values based on windowing
        min_val, max_val = wc - ww // 2, wc + ww // 2
        # Clip pixel values between min and max and normalize to a range of 0-255
        img_data = np.clip(img_data, min_val, max_val)
        # Return the windowed image data as 8-bit unsigned integer values
        return ((img_data - min_val) / (max_val - min_val) * 255).astype(np.uint8)
    except Exception as e:
        # Print error message if windowing fails
        print(f"Error in apply_windowing: {e}")
        return None

# Convert a DICOM file to a PNG file
def convert_dicom_to_png(input_file, output_file):
    try:
        # Read DICOM file
        ds = pydicom.dcmread(input_file)
        # Extract pixel data from the DICOM file
        img_data = ds.pixel_array
        # Get the Window Center (WC) and Window Width (WW) from DICOM metadata
        wc = ds.get("WindowCenter", img_data.mean())
        ww = ds.get("WindowWidth", img_data.max() - img_data.min())
        # Apply windowing to the image
        img_data = apply_windowing(img_data, wc if isinstance(wc, (int, float)) else wc[0], ww if isinstance(ww, (int, float)) else ww[0])
        # If windowing was successful, save the image as a PNG
        if img_data is not None:
            image = Image.fromarray(img_data)  # Convert to PIL Image object
            image.save(output_file)  # Save as PNG
    except Exception as e:
        # Print error message if DICOM to PNG conversion fails
        print(f"Error converting DICOM to PNG: {e}")

# Check if the image is light (average pixel value > 128)
def is_image_light(image_path):
    try:
        # Open the image file
        image = Image.open(image_path)
        # Convert the image to grayscale
        image_gray = ImageOps.grayscale(image)
        # Convert the grayscale image to a NumPy array
        image_array = np.array(image_gray)
        # Calculate the average pixel value of the grayscale image
        avg_pixel_value = np.mean(image_array)
        # Return True if the average pixel value is greater than 128, indicating a light image
        return avg_pixel_value > 128
    except Exception as e:
        # Print error message if the lightness check fails
        print(f"Error in checking image lightness: {e}")
        return False

# Process an image: convert DICOM to PNG, invert if light, and delete the DICOM file
def process_image(dicom_path, png_path):
    try:
        # Convert DICOM file to PNG
        convert_dicom_to_png(dicom_path, png_path)
        # If the PNG file exists and the image is light, invert the colors
        if os.path.exists(png_path) and is_image_light(png_path):
            image = Image.open(png_path)  # Open the PNG image
            image_inverted = ImageOps.invert(image.convert('RGB'))  # Invert the image
            image_inverted.save(png_path)  # Save the inverted image
        # Remove the original DICOM file after processing
        os.remove(dicom_path)
    except Exception as e:
        # Print error message if image processing fails
        print(f"Error processing image {dicom_path}: {e}")

# Process all DICOM images in a folder
def process_images_in_folder(folder_path):
    # Use ThreadPoolExecutor to process images in parallel with 4 threads
    with ThreadPoolExecutor(max_workers=4) as executor:  # Adjust the number of workers as needed
        futures = []  # List to store future tasks
        # Iterate through each subfolder in the specified folder
        for subfolder in tqdm(os.listdir(folder_path)):
            subfolder_path = os.path.join(folder_path, subfolder)  # Full path of the subfolder
            if os.path.isdir(subfolder_path):  # Check if it's a directory
                # Iterate through each file in the subfolder
                for file_name in os.listdir(subfolder_path):
                    # Process only DICOM files (.dcm)
                    if file_name.lower().endswith('.dcm'):
                        dicom_path = os.path.join(subfolder_path, file_name)  # Full path of the DICOM file
                        png_path = os.path.join(subfolder_path, file_name.replace('.dcm', '.png'))  # Path for the PNG file
                        # Submit the process_image task to the thread pool
                        futures.append(executor.submit(process_image, dicom_path, png_path))
        # Wait for all the tasks to complete
        for future in tqdm(futures):
            try:
                future.result()  # Retrieve the result of each task
            except Exception as e:
                # Print error message if any task fails
                print(f"Error in processing a future: {e}")

if __name__ == "__main__":
    # Specify the folder containing the DICOM files
    folder_path = r"C:\Users\zafer\Desktop\competition_dataset\competition_dataset"
    # Process the DICOM files in the folder
    process_images_in_folder(folder_path)
    # Print a message once processing is complete
    print("Image processing complete.")


# Functions

In [ ]:
import cv2

# Function to predict labels for a given frame using the model
def Predict_frame(frame, model):
    """
    Predict bounding boxes and class labels for a given frame.
    
    Parameters:
    - frame: The input image/frame.
    - model: The YOLO model used for prediction.
    
    Returns:
    - A list of predicted labels [class, probability, x_center, y_center, width, height] for each detection.
    """
    predictions = model(frame, save_txt=None)  # Get predictions from the model without saving to a file
    labels = []  # Initialize an empty list to store labels
    for idx, prediction in enumerate(predictions[0].boxes.xywhn):  # xywhn is the normalized bounding box
        cls = int(predictions[0].boxes.cls[idx].item())  # Get the class of the detected object
        prob = predictions[0].boxes.conf[idx].item()  # Get the confidence score of the prediction
        # Format the prediction into a string: class, probability, x_center, y_center, width, height
        line = f"{cls} {prob} {prediction[0].item()} {prediction[1].item()} {prediction[2].item()} {prediction[3].item()}"
        numbers = list(map(float, line.split()))  # Convert the string to a list of floats
        labels.append(numbers)  # Append the label to the list
    return labels  # Return the list of labels

# Function to unnormalize YOLO format coordinates into pixel-based coordinates
def unnormalize_Yolo_cords(label, image):
    """
    Convert YOLO format bounding box coordinates into pixel-based coordinates.

    Parameters:
    - label: YOLO format [class, probability, x_center, y_center, width, height].
    - image: The image/frame to get dimensions from.

    Returns:
    - The class, probability, and pixel-based bounding box coordinates [x1, y1, x2, y2].
    """
    img_height, img_width, _ = image.shape  # Get image dimensions
    
    cls, prob, x_center, y_center, width_bbox, height_bbox = label
    x = x_center * img_width
    y = y_center * img_height
    bbox_width = width_bbox * img_width
    bbox_height = height_bbox * img_height

    # Calculate pixel-based coordinates
    x1 = int(x - bbox_width / 2)
    x2 = int(x + bbox_width / 2)
    y1 = int(y - bbox_height / 2)
    y2 = int(y + bbox_height / 2)

    return cls, prob, x1, y1, x2, y2

# Function to reverse the transformation and retrieve original label coordinates
def reverse_label_conf(x1, y1, x2, y2, image):
    """
    Reverse the transformation to get the original label coordinates.
    
    Parameters:
    - x1, y1, x2, y2: Transformed bounding box coordinates.
    - image: The image to derive the width and height from.
    
    Returns:
    - Original coordinates as a formatted string: "x1,y1;x1,y2;x2,y2;x2,y1".
    """
    # Convert image to RGB format (if necessary)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    height, width, _ = image_rgb.shape

    half_width = width / 2
    half_height = height / 2
    
    # Reverse transformation
    x1_orig = round(x1 - half_width, 4)
    y1_orig = round(y1 - half_height, 5)
    x2_orig = round(x2 - half_width, 4)
    y2_orig = round(y2 - half_height, 5)
    
    # Format into a string
    coord_str = f"{x1_orig},{y1_orig};{x1_orig},{y2_orig};{x2_orig},{y2_orig};{x2_orig},{y1_orig}"
    
    return coord_str

# Example usage for testing:
# transformed_coords = (480, 1085, 742, 1380)  # Sample transformed coordinates
# original_labels = reverse_label_conf(*transformed_coords, image)  # Get original coordinates
# print(original_labels)  # Print the original label coordinates


# Giving the image to the model and testing








In [ ]:
# Import necessary libraries
import cv2
import os
import pandas as pd

# Directory containing your images
image_dir = r'C:\Users\zafer\Desktop\KIitlekals_competition_dataset - Kopya\competition_dataset'  # Change this to your image directory

# Lists to store DataFrame rows
all_data = []

# Iterate through all image files in the directory
for image_file in os.listdir(image_dir):
    if image_file.lower().endswith(('.jpg', '.jpeg', '.png')):  # Process only image files
        # Construct the full image path
        image_path = os.path.join(image_dir, image_file)
        
        # Read and preprocess the image
        image_with_orginal_size = cv2.imread(image_path)  # Read the image with its original size
        imagefor_predict = cv2.resize(image_with_orginal_size, (1024, 1024))  # Resize image for model prediction
        
        # Predict the labels using the trained model
        labels = Predict_frame(frame=imagefor_predict, model=kitle_kalsifikasyon)  # Prediction function
        
        # Initialize lists for storing results
        kitle = []  # List for storing mass (kitle) bounding boxes
        kalsifikasyon = []  # List for storing calcification bounding boxes
        kitle_guven = []  # List for storing mass confidence scores
        kalsifikasyon_guven = []  # List for storing calcification confidence scores
        
        # Process each label returned by the model
        for i in range(len(labels)):
            print(i)  # Print the index to track the loop progress
            # Get the bounding box coordinates and other details in pixel format
            cls, prob, x1, y1, x2, y2 = unnormalize_Yolo_cords(label=labels[i], image=image_with_orginal_size)
            
            # Convert transformed coordinates back to original format
            transformed_coords = (x1, y1, x2, y2)
            original_labels = reverse_label_conf(*transformed_coords, image=image_with_orginal_size)  # Reverse transformation
            
            # Append labels and confidence based on class (0 for mass, 1 for calcification)
            if cls == 0:
                kitle.append([original_labels])  # Append mass bounding box
                kitle_guven.append([prob])  # Append mass confidence score
            else:
                kalsifikasyon.append([original_labels])  # Append calcification bounding box
                kalsifikasyon_guven.append([prob])  # Append calcification confidence score
            
            print(original_labels)  # Print the original labels for debugging
        
        # Ensure all lists are structured as list of lists
        kitle = kitle if kitle else []  # Ensure 'kitle' is not empty
        kitle_guven = kitle_guven if kitle_guven else []  # Ensure 'kitle_guven' is not empty
        kalsifikasyon = kalsifikasyon if kalsifikasyon else [[]]  # If empty, assign a placeholder
        kalsifikasyon_guven = kalsifikasyon_guven if kalsifikasyon_guven else [[]]  # Same for confidence scores
        
        # Create a dictionary for the current image
        image_data = {
            'filename': image_file,  # Store the filename of the image
            'kitle': kitle,  # Store mass bounding boxes
            'kitle_guven': kitle_guven,  # Store mass confidence scores
            'kalsifikasyon': kalsifikasyon,  # Store calcification bounding boxes
            'kalsifikasyon_guven': kalsifikasyon_guven  # Store calcification confidence scores
        }
        
        # Append the current image data to the list
        all_data.append(image_data)  # Add this image's data to the global list

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_data)  # Convert the list of dictionaries to a pandas DataFrame

# Save the DataFrame to an Excel file
df.to_excel(r'C:\Users\zafer\Desktop\kitle_kals_image_data.xlsx', index=False)  # Save DataFrame to Excel

print("DataFrame successfully saved to image_data.xlsx")  # Print confirmation message
